# Trabalho FINAL — Classificação de Revisões de Filmes
- Equipe: Pioneiros de Dados
  - clebr
  - vi...

## Instalação das Bibliotecas

In [1]:
!pip install catboost
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes     import MultinomialNB
from sklearn.metrics         import accuracy_score ,balanced_accuracy_score
from sklearn.ensemble        import RandomForestClassifier

import nltk
from nltk.corpus import stopwords

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling  import RandomOverSampler

In [2]:
#!pip install catboost
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils.class_weight       import compute_class_weight


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


#Acesso ao drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from imblearn.over_sampling import SMOTE
from collections import Counter

from nltk.stem import PorterStemmer
import spacy

## Carregar e preparar os dados

In [5]:
train_critic = pd.read_csv("file:///content/drive/MyDrive/Colab Notebooks/INF_0619/train_audience.csv", lineterminator='\n')
test_critic  = pd.read_csv("file:///content/drive/MyDrive/Colab Notebooks/INF_0619/test_audience.csv" , lineterminator='\n')

Preparando dados - retirando review nulos

In [6]:
train_critic = train_critic[train_critic['Review'].notna() & train_critic['Review'].str.strip().astype(bool)]
test_critic  = test_critic [test_critic ['Review'].notna() & test_critic ['Review'].str.strip().astype(bool)]

In [7]:
train_critic['Review'] = train_critic['Review'].apply(lambda x: x.lower())
test_critic ['Review'] = test_critic ['Review'].apply(lambda x: x.lower())

stemmer = PorterStemmer()
nlp = spacy.load('en_core_web_sm')

# Funçoes
def apply_stemming(text):
    tokens = word_tokenize(text)
    stemmed = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed)
def apply_lemmatization(text):
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc]
    return ' '.join(lemmatized)

# Aplicar as transformações na coluna Review
train_critic['Review'] = train_critic['Review'].dropna().apply(apply_lemmatization)
test_critic['Review'] = test_critic['Review'].dropna().apply(apply_lemmatization)

# Amostragem

In [8]:
data = train_critic

X = data[['Show', 'Review']]
y = data['Rating'].astype(str)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
##########################################################################################################################################

# SMOTE

# Convert text data to numerical representation using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=100, max_df=0.85, min_df=10, ngram_range=(1, 3))

X_train_vec   = vectorizer.fit_transform(X_train['Review'])
X_valid_smote = vectorizer.    transform(X_valid['Review'])

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_vec, y_train)

# volumetrias
print('\nPré balanceamento')
print(Counter(y))
print('\nPós balanceamento')
print('Treino:    ',Counter(y_train_smote))
print('Validação: ',Counter(y_valid      ))


Pré balanceamento
Counter({'5.0': 23642, '0.5': 7016, '4.0': 3869, '4.5': 3792, '1.0': 3309, '2.0': 2690, '3.5': 2340, '3.0': 2141, '1.5': 2040, '2.5': 1577})

Pós balanceamento
Treino:     Counter({'0.5': 18970, '5.0': 18970, '1.0': 18970, '4.0': 18970, '2.5': 18970, '2.0': 18970, '3.0': 18970, '4.5': 18970, '1.5': 18970, '3.5': 18970})
Validação:  Counter({'5.0': 4672, '0.5': 1445, '4.0': 799, '4.5': 766, '1.0': 658, '2.0': 534, '3.5': 463, '3.0': 433, '1.5': 392, '2.5': 322})


In [10]:
##########################################################################################################################################

# SIMPLE UNDERSAMPLE

# Converter X para índices porque Random*Sampler trabalha com arrays/matrizes
X_indices = X_train.index.values.reshape(-1, 1)

# Inicializar sampler
sampler = RandomUnderSampler(random_state=42)

# Aplicar o *sample
_, y_resampled_indices = sampler.fit_resample(X_indices, y_train)

# Filtrar o DataFrame original com os índices resampleados
balanced_data = data.iloc[y_resampled_indices.to_numpy().flatten().astype(float)]

# Divide train e test usando somente o conjunto train_critic
X_train_under = balanced_data[['Show', 'Review']]
y_train_under = balanced_data['Rating']


# volumetrias
print('\nPré balanceamento')
print(Counter(data['Rating']))
print('\nPós balanceamento')
print('Treino:    ',Counter(y_train_under))
print('Validação: ',Counter(y_valid      ))


Pré balanceamento
Counter({5.0: 23642, 0.5: 7016, 4.0: 3869, 4.5: 3792, 1.0: 3309, 2.0: 2690, 3.5: 2340, 3.0: 2141, 1.5: 2040, 2.5: 1577})

Pós balanceamento
Treino:     Counter({2.5: 2510, 3.0: 2510, 2.0: 2510, 4.5: 2510, 0.5: 1255, 5.0: 1255})
Validação:  Counter({'5.0': 4672, '0.5': 1445, '4.0': 799, '4.5': 766, '1.0': 658, '2.0': 534, '3.5': 463, '3.0': 433, '1.5': 392, '2.5': 322})


In [11]:
##########################################################################################################################################

# SIMPLE UNDERSAMPLE

# Converter X para índices porque Random*Sampler trabalha com arrays/matrizes
X_indices = X_train.index.values.reshape(-1, 1)

# Inicializar sampler
sampler = RandomOverSampler(random_state=42)

# Aplicar o *sample
_, y_resampled_indices = sampler.fit_resample(X_indices, y_train)

# Filtrar o DataFrame original com os índices resampleados
balanced_data = data.iloc[y_resampled_indices.to_numpy().flatten().astype(float)]

# Divide train e test usando somente o conjunto train_critic
X_train_over = balanced_data[['Show', 'Review']]
y_train_over = balanced_data['Rating']

# volumetrias
print('\nPré balanceamento')
print(Counter(data['Rating']))
print('\nPós balanceamento')
print('Treino:    ',Counter(y_train_over))
print('Validação: ',Counter(y_valid      ))


Pré balanceamento
Counter({5.0: 23642, 0.5: 7016, 4.0: 3869, 4.5: 3792, 1.0: 3309, 2.0: 2690, 3.5: 2340, 3.0: 2141, 1.5: 2040, 2.5: 1577})

Pós balanceamento
Treino:     Counter({2.5: 37940, 4.5: 37940, 3.0: 37940, 2.0: 37940, 0.5: 18970, 5.0: 18970})
Validação:  Counter({'5.0': 4672, '0.5': 1445, '4.0': 799, '4.5': 766, '1.0': 658, '2.0': 534, '3.5': 463, '3.0': 433, '1.5': 392, '2.5': 322})


In [12]:
##########################################################################################################################################
import numpy as np
# PESOS POPULACIONAIS

classes = y_train.to_frame()['Rating'].unique()

weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(classes),
    y=y_train.to_frame()['Rating']
)

class_weights = dict(zip(classes, weights))
class_weights

{'0.5': 0.7526835397594687,
 '5.0': 0.2210437532946758,
 '1.0': 1.5817427385892115,
 '4.0': 1.3658631921824105,
 '2.5': 3.341195219123506,
 '2.0': 1.9448979591836735,
 '3.0': 2.4550351288056205,
 '4.5': 1.3857237276933245,
 '1.5': 2.5444174757281552,
 '3.5': 2.233990410229089}

# Vetorizando

In [13]:
for i in ['_under','_over','']:

  vectorizer = TfidfVectorizer(stop_words='english', max_features=100, max_df=0.85, min_df=10, ngram_range=(1, 3))

  globals()['X_train%s'%i] = vectorizer.fit_transform(globals()['X_train%s'%i]['Review'])
  globals()['X_valid%s'%i] = vectorizer.    transform(X_valid['Review'])

# Modelos

In [16]:
# Dicionário com modelos
models = {
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=10, max_features=2),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'SVC(gamma=2, C=1)': SVC(gamma=2, C=1),
    'SVC(kernel="linear",C=0.025,probability=True)': SVC(kernel="linear",C=0.025,probability=True),
    }

# Dicionário com modelos
model_com_peso = {
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=10, max_features=2, class_weight = class_weights),
    'SVC(gamma=2, C=1)': SVC(gamma=2, C=1, class_weight = class_weights),
    'SVC(kernel="linear",C=0.025,probability=True)': SVC(kernel="linear",C=0.025,probability=True, class_weight = class_weights),
    }

# Treinar

Smote

In [ ]:
results = []

for name, model in models.items():
    model.fit(X_train_smote, y_train_smote.astype(str))
    print(name, balanced_accuracy_score(y_train_smote.astype(str), model.predict(X_train_smote)), balanced_accuracy_score(y_valid, model.predict(X_valid_smote)))
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train_smote.astype(str), model.predict(X_train_smote))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid_smote))})

# Exibir resultados em um DataFrame
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='Validação', ascending=False))

Random Forest 0.9061043753294676 0.16012075981924728
K-Nearest Neighbors 0.7999472851871376 0.15661468128459913


Simple undersample

In [ ]:
results = []

for name, model in models.items():
    model.fit(X_train_under, y_train_under.astype(str))
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train_under.astype(str), model.predict(X_train_under))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid_under))})

# Exibir resultados em um DataFrame
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='Validação', ascending=False))

                                           Model  Treino  Validação
2  SVC(kernel="linear",C=0.025,probability=True)     1.0   0.102727
1                              SVC(gamma=2, C=1)     1.0   0.095366
3                            K-Nearest Neighbors     1.0   0.095366
0                                  Random Forest     1.0   0.093076


Simple oversample

In [ ]:
results = []

for name, model in models.items():
    model.fit(X_train_over, y_train_over.astype(str))
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train_over.astype(str), model.predict(X_train_over))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid_over))})

# Exibir resultados em um DataFrame
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='Validação', ascending=False))

                                           Model  Treino  Validação
2  SVC(kernel="linear",C=0.025,probability=True)     1.0   0.102727
1                              SVC(gamma=2, C=1)     1.0   0.094146
3                            K-Nearest Neighbors     1.0   0.094146
0                                  Random Forest     1.0   0.093420


Peso populacional

In [15]:
results = []

for name, model in model_com_peso.items():
    model.fit(X_train, y_train.astype(str))
    results.append({  'Model'    : name
                    , 'Treino'   : balanced_accuracy_score(y_train.astype(str), model.predict(X_train))
                    , 'Validação': balanced_accuracy_score(y_valid, model.predict(X_valid))})

# Exibir resultados em um DataFrame
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='Validação', ascending=False))

Random Forest 0.8421151670182183 0.1480114279771004
SVC(gamma=2, C=1) 0.7739113371687381 0.20002525969537147
SVC(kernel="linear",C=0.025,probability=True) 0.2197160596535494 0.19360658599379227
                                           Model    Treino  Validação
1                              SVC(gamma=2, C=1)  0.773911   0.200025
2  SVC(kernel="linear",C=0.025,probability=True)  0.219716   0.193607
0                                  Random Forest  0.842115   0.148011
